In [1]:
import numpy as np
from itertools import product

In [2]:
def _contains_duplicates(X):
    return np.sum(np.unique(X)) != np.sum(X)

def contains_duplicates(sol):
    return any(_contains_duplicates(sol[r,:]) for r in range(9)) or \
           any(_contains_duplicates(sol[:,r]) for r in range(9)) or \
           any(_contains_duplicates(sol[r:r+3:,c:c+3]) for r in range(0,9,3) for c in range(0,9,3))

def valid_solution(sol):
    return not contains_duplicates(sol) and np.sum(sol) == (1+2+3+4+5+6+7+8+9) * 9

def print_sudoku(sudoku):
    print("+-------+-------+-------+")
    for b in range(0, 9, 3):
        for r in range(3):
            print("|", " | ".join(" ".join(str(_) for _ in sudoku[b+r, c:c+3]) for c in range(0, 9, 3)), "|")
        print("+-------+-------+-------+")

In [3]:
def trim(possible_values, links, r):
    trimmed_columns = []
    for j in links[r]:
        for i in possible_values[j]:
            for k in links[i]:
                if k != j:
                    possible_values[k].remove(i)
        trimmed_columns.append(possible_values.pop(j))
    return trimmed_columns

def expand(possible_values, links, r, trimmed_columns):
    expanded = False
    for j in reversed(links[r]):
        possible_values[j] = trimmed_columns.pop()
        for i in possible_values[j]:
            for k in links[i]:
                if k != j:
                    possible_values[k].add(i)
                    expanded = True
    return expanded

def search(possible_values, links, solution, num_nodes):

    #? If we have run out of possibilites we could have found a solution
    if not possible_values:
        return solution
    else:
        #? Pick the cell with least possibilities to prevent massive branching (i.e. the cell with least number of ones)
        col = min(possible_values, key=lambda c: len(possible_values[c]))

        for r in list(possible_values[col]):
            solution.append(r)

            #? When we select a column we need to eliminate all rows intersecting by the choice made
            trimmed_columns = trim(possible_values, links, r)

            s = search(possible_values, links, solution, num_nodes)
            if s:
                return s

            if expand(possible_values, links, r, trimmed_columns):
                num_nodes[0] += 1
            solution.pop()

def solve(sudoku):

    num_nodes = [1] #? Nr. of nodes expanded
    root = sudoku.copy()

    #! Initialization (transform the 2D array into a graph via adjacency lists)
    links = {}
    for row, col, n in product(range(9), range(9), range(1, 9+1)):
        box = (row//3)*3 + (col//3)
        links[(row, col, n)] = [('rc', (row, col)), ('rn', (row, n)), ('cn', (col, n)), ('bn', (box, n))]

    possible_values = ([('rc', x) for x in product(range(9), range(9))] + [('rn', x) for x in product(range(9), range(1, 9+1))] +
                       [('cn', x) for x in product(range(9), range(1, 9+1))] + [('bn', x) for x in product(range(9), range(1, 9+1))])

    possible_values = { k : set() for k in possible_values }
    for i, row in links.items():
        for j in row:
            possible_values[j].add(i)
    
    #! Trim illegal moves based on initial cells values
    for r, row in enumerate(sudoku):
        for c, n in enumerate(row):
            if n:
                trim(possible_values, links, (r,c,n))
    
    #! Start searching for a solution
    solution = search(possible_values, links, [], num_nodes)

    if solution:

        for (row, col, n) in solution:
            root[row][col] = n

        if valid_solution(root):
            print(f'Solved after expanding {num_nodes[0]:,} nodes')
            return root

    print(f'Giving up after expanding {num_nodes[0]:,} nodes')
    return None

In [4]:
def sudoku_generator(sudokus=1, *, kappa=5, random_seed=None):
    if random_seed:
        np.random.seed(random_seed)
    for puzzle in range(sudokus):
        sudoku = np.zeros((9, 9), dtype=np.int8)
        for cell in range(np.random.randint(kappa)):
            for p, val in zip(np.random.randint(0, 8, size=(9, 2)), range(1, 10)):
                tmp = sudoku.copy()
                sudoku[tuple(p)] = val
                if contains_duplicates(sudoku):
                    sudoku = tmp
        yield sudoku.copy()

In [5]:
diabolical_sudoku = np.array([
    [8, 0, 0,    0, 0, 0,    0, 0, 0],
    [0, 0, 3,    6, 0, 0,    0, 0, 0],
    [0, 7, 0,    0, 9, 0,    2, 0, 0],
    
    [0, 5, 0,    0, 0, 7,    0, 0, 0],
    [0, 0, 0,    0, 4, 5,    7, 0, 0],
    [0, 0, 0,    1, 0, 0,    0, 3, 0],
    
    [0, 0, 1,    0, 0, 0,    0, 6, 8],
    [0, 0, 8,    5, 0, 0,    0, 1, 0],
    [0, 9, 0,    0, 0, 0,    4, 0, 0]
], dtype=np.int8)

In [6]:
for sudoku in sudoku_generator(random_seed=42):
    print_sudoku(sudoku)
    solution = solve(sudoku)
    if solution is not None:
        print_sudoku(solution)

+-------+-------+-------+
| 0 0 1 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 3 0 |
| 0 0 6 | 8 0 0 | 5 2 0 |
+-------+-------+-------+
| 9 7 0 | 4 0 0 | 0 8 0 |
| 6 0 0 | 0 3 0 | 1 0 0 |
| 0 0 0 | 0 8 6 | 0 0 0 |
+-------+-------+-------+
| 0 4 0 | 9 0 0 | 0 0 0 |
| 0 0 9 | 5 7 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
Solved after expanding 1 nodes
+-------+-------+-------+
| 7 8 1 | 3 2 5 | 4 9 6 |
| 4 2 5 | 6 9 7 | 8 3 1 |
| 3 9 6 | 8 1 4 | 5 2 7 |
+-------+-------+-------+
| 9 7 2 | 4 5 1 | 6 8 3 |
| 6 5 8 | 7 3 9 | 1 4 2 |
| 1 3 4 | 2 8 6 | 7 5 9 |
+-------+-------+-------+
| 5 4 7 | 9 6 2 | 3 1 8 |
| 8 1 9 | 5 7 3 | 2 6 4 |
| 2 6 3 | 1 4 8 | 9 7 5 |
+-------+-------+-------+


In [7]:
solution = solve(diabolical_sudoku)
if solution is not None:
    print_sudoku(solution)

Solved after expanding 258 nodes
+-------+-------+-------+
| 8 1 2 | 7 5 3 | 6 4 9 |
| 9 4 3 | 6 8 2 | 1 7 5 |
| 6 7 5 | 4 9 1 | 2 8 3 |
+-------+-------+-------+
| 1 5 4 | 2 3 7 | 8 9 6 |
| 3 6 9 | 8 4 5 | 7 2 1 |
| 2 8 7 | 1 6 9 | 5 3 4 |
+-------+-------+-------+
| 5 2 1 | 9 7 4 | 3 6 8 |
| 4 3 8 | 5 2 6 | 9 1 7 |
| 7 9 6 | 3 1 8 | 4 5 2 |
+-------+-------+-------+
